In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
plt.style.use('RW_visualization.mplstyle')


In [ ]:
# # import warnings filter
# from pandas.core.common import SettingWithCopyWarning
# import warnings
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)


# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


Load data

In [ ]:
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Remove all vaccine rows
vaccine_detail = vaccine_detail[~(vaccine_detail['vaccine']=='All vaccine')]

Calculate vaccine efficacy and CIs by relative risk and poission regression with robust error variance if no 0 in each group

In [ ]:
# Calculate vaccine efficacy and confidence interval
RW_vaccine_efficacy = np.ones(len(vaccine_detail))*np.nan
RW_lower = np.ones(len(vaccine_detail))*np.nan
RW_higher = np.ones(len(vaccine_detail))*np.nan
for i, row in vaccine_detail.iterrows():
    xv = row['no_of_participants_in_vaccine_group']
    xp = row['no_of_participants_in_placebo_groupcontrol_group']
    xve = row['no_of_infected_in_vaccine_group']
    xpe = row['no_of_infected_in_placebo_group']
    if xve != 'X':
        # Relative risk
        r1 = xve/xv
        r2 = xpe/xp
        theta = r1/r2
        VE = 1-theta
        RW_vaccine_efficacy[i] = VE
        if (xve != 0) & (xpe != 0):
            # Poisson regression with robust error variance
            variance = 1/xve - 1/xv + 1/xpe - 1/xp
            se = np.sqrt(variance)
            RW_lower[i] = 1-theta*np.exp(1.96*se)
            RW_higher[i] = 1-theta*np.exp(-1.96*se)
        else:
            RW_lower[i] = 0
            RW_higher[i] = 0


In [ ]:
# Append my result to the table
vaccine_detail['RW_efficacy_in_%'] = RW_vaccine_efficacy*100
vaccine_detail['RW_lower'] = RW_lower*100
vaccine_detail['RW_upper'] = RW_higher*100

vaccine_detail.to_excel('rw_vaccine_details.xlsx')


In [ ]:
# Extract references array
import re
references = np.array([])
for i in vaccine_detail['reference'].str.split('_'):
    reference = i[0]
    reference = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " et al. ", reference)
    references = np.append(references, reference)

In [ ]:
# Correct article's vaccine efficacy label
for i in range(len(vaccine_detail)):
    efficacy_method = vaccine_detail.method_to_calculate_ve.loc[i]
    for j, word in enumerate(efficacy_method):
        if word.isalpha() == True:
            efficacy_method = efficacy_method[j::]
            efficacy_method = efficacy_method[0].upper() + efficacy_method[1::]
            vaccine_detail.method_to_calculate_ve.loc[i] = efficacy_method
            break

paper_ve_methods = vaccine_detail.method_to_calculate_ve.to_numpy()
paper_ve_methods[paper_ve_methods == 'X'] = 'Not specify'

In [ ]:
# Correct article's CI label
for i in range(len(vaccine_detail)):
    CI = vaccine_detail.method_to_calculate_ci.loc[i]
    CI = CI[0].upper() + CI[1::]
    vaccine_detail.method_to_calculate_ci.loc[i] = CI

paper_ci_methods = vaccine_detail.method_to_calculate_ci.to_numpy()
paper_ci_methods[paper_ci_methods == 'X'] = 'Not specify'
paper_ci_methods[paper_ci_methods ==
                 "Stratified Cox proportional-hazards model with Efron's method of tie handling"] = 'Cox proportional-hazards'
paper_ci_methods[paper_ci_methods == 'Poisson regression with robust error variance'] = 'Poisson regression'

Plot my CIs vs paper's CIs

In [ ]:
vaccine_detail.columns


In [ ]:
def plot_compare_reproduced_vaccine_efficacy_scalar_plot(paper_vaccine_efficacy, paper_lower_bound,
                                                         paper_upper_bound, paper_ve_methods, paper_ci_methods,
                                                         RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound, save_fig=True):

    plt.plot()
    for i in range(len(paper_vaccine_efficacy)):
        if paper_lower_bound[i] == 0 and paper_upper_bound[i] == 0:
            x_error = np.array([[0, 0]]).T
        else:
            x_error = np.array([[paper_vaccine_efficacy[i]-paper_lower_bound[i],
                                paper_upper_bound[i]-paper_vaccine_efficacy[i]]]).T

        x_error[x_error<0] = 0

        if RW_lower_bound[i]==0 and RW_upper_bound[i]==0:
            y_error = np.array([[0, 0]]).T
        else:
            y_error = np.array([[RW_vaccine_efficacy[i]-RW_lower_bound[i],
                                RW_upper_bound[i]-RW_vaccine_efficacy[i]]]).T

        plt.errorbar(paper_vaccine_efficacy[i], RW_vaccine_efficacy[i],
                     xerr=x_error, yerr=y_error, fmt='.', color='k', elinewidth=0.5, capsize=0)
    plt.plot([0, 105], [0, 105], '--k')
    plt.xlabel(
        'Reported vaccine efficacy with $95\%$ CIs in the original study')
    plt.ylabel('Vaccine efficacy with $95\%$ CIs based on relative risk and \n Poisson regression with robust error')
    plt.grid()
    plt.xlim(0, 105)
    plt.ylim(0, 105)

    if save_fig == True:
        plt.savefig('RW_vaccine_efficacy_reproduction.pdf')


In [ ]:
def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique), numpoints=1)

In [ ]:
def plot_compare_reproduced_vaccine_efficacy_error_bar_plot(paper_vaccine_efficacy, paper_lower_bound,
                                                            paper_upper_bound, paper_ve_methods, paper_ci_methods,
                                                            RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound,
                                                            references, save_fig=True):
    # Initialization
    palette_all = sb.color_palette(
        'Set1', n_colors=len(paper_vaccine_efficacy)+2)
    # Remove yellow
    palette = palette_all[0:5] + palette_all[6:14]
    for i in range(len(palette_all)//len(palette)):
        palette = palette + palette[0:13]
    palette = palette[0:len(paper_vaccine_efficacy)]

    # Sort
    sort_index = np.argsort(paper_vaccine_efficacy)
    # sort_index = sort_index[::-1]
    paper_vaccine_efficacy = paper_vaccine_efficacy[sort_index]
    paper_lower_bound = paper_lower_bound[sort_index]
    paper_upper_bound = paper_upper_bound[sort_index]
    paper_ve_methods = paper_ve_methods[sort_index]
    paper_ci_methods = paper_ci_methods[sort_index]
    RW_vaccine_efficacy = RW_vaccine_efficacy[sort_index]
    RW_lower_bound = RW_lower_bound[sort_index]
    RW_upper_bound = RW_upper_bound[sort_index]
    references = references[sort_index]

    # Markers and colors
    all_markers = np.array(
        ['o', 'v', '^', '<', '>', 's', 'P', 'p', '*', 'h', 'H', 'X', 'D'])
    ve_method_markers_dict = dict(
        zip(np.unique(paper_ve_methods), all_markers[0:len(np.unique(paper_ve_methods))]))
    ve_method_color_dict = dict(
        zip(np.unique(paper_ve_methods), palette[0:len(np.unique(paper_ve_methods))]))

    # Confidence interval and color
    ci_method_color_dict = dict(
        zip(np.unique(paper_ci_methods), palette[0:len(np.unique(paper_ci_methods))]))

    # Plot
    index = 0
    index_array = np.array([])
    fig = plt.figure(figsize=(11, len(paper_vaccine_efficacy)*0.4))
    ax = fig.subplots(1, 1)
    for i in range(len(sort_index)):
        efficacy = np.array(
            [paper_vaccine_efficacy[i], RW_vaccine_efficacy[i]])
        lower_bounds = np.array([paper_lower_bound[i], RW_lower_bound[i]])
        upper_bounds = np.array([paper_upper_bound[i], RW_upper_bound[i]])
        paper_ve_method = paper_ve_methods[i]
        paper_ci_method = paper_ci_methods[i]
        
        if lower_bounds[0]==0 and upper_bounds[0]==0:
            xerr0 = np.array([[0, 0]]).T
        else:
            xerr0 = np.array([[paper_vaccine_efficacy[i]-paper_lower_bound[i],
                               paper_upper_bound[i]-paper_vaccine_efficacy[i]]]).T

        xerr0[xerr0<0] = 0
        if lower_bounds[1]==0 and upper_bounds[1]==0:
            xerr1 = np.array([[0, 0]]).T
        else:
            xerr1 = np.array([[RW_vaccine_efficacy[i]-RW_lower_bound[i],
                               RW_upper_bound[i]-RW_vaccine_efficacy[i]]]).T
        
        line1 = ax.errorbar(efficacy[0], index, xerr=xerr0,
                                                fmt=ve_method_markers_dict[paper_ve_method],
                                                ms=5, ecolor=ci_method_color_dict[paper_ci_method], 
                                                markerfacecolor=ve_method_color_dict[paper_ve_method], 
                                                label=paper_ve_method+'\n'+paper_ci_method)
        line2 = ax.errorbar(efficacy[1], index+1, xerr=xerr1, 
                                                    fmt=ve_method_markers_dict['Relative risk'], 
                                                    ms=5, 
                                                    ecolor=ci_method_color_dict['Poisson regression'], 
                                                    markerfacecolor=ve_method_color_dict['Relative risk'])
        index += 4
        index_array = np.append(index_array, index)
    handles, labels = ax.get_legend_handles_labels()
    # remove the errorbars
    # handles = [h[0] for h in handles]
    # use them in the legend
    # ax.legend(handles[0:2], labels[0:2], loc='upper right', numpoints=1)
    legend_without_duplicate_labels(ax)
    # ax.legend(loc='upper right', numpoints=1)

    plt.xticks(np.arange(0, 110, 10))
    plt.yticks(index_array-4, references, fontsize=20)
    plt.ylim([-1, index])
    plt.xlim([0, 200])
    plt.gca().invert_yaxis()
    plt.gca().yaxis.grid()
    plt.gca().xaxis.grid()
    plt.xlabel('Vaccine efficacy with $95\%$ CIs', fontsize=20)

    if save_fig == True:
        plt.savefig('RW_vaccine_efficacy_reproduction_error_bar.pdf')


In [ ]:
# Clean nan
RW_vaccine_efficacy = vaccine_detail['RW_efficacy_in_%'].to_numpy()
nan_map = np.isnan(RW_vaccine_efficacy)
RW_vaccine_efficacy = RW_vaccine_efficacy[~nan_map]

paper_vaccine_efficacy = vaccine_detail['efficacy_in_%'].to_numpy()
paper_vaccine_efficacy = paper_vaccine_efficacy[~nan_map]


paper_lower_bound = vaccine_detail['lower'].to_numpy()
paper_lower_bound[paper_lower_bound == 'X'] = 0
paper_lower_bound = paper_lower_bound[~nan_map]

paper_upper_bound = vaccine_detail['upper'].to_numpy()
paper_upper_bound[paper_upper_bound == 'X'] = 0
paper_upper_bound = paper_upper_bound[~nan_map]

RW_vaccine_efficacy = vaccine_detail['RW_efficacy_in_%'].to_numpy()
RW_vaccine_efficacy = RW_vaccine_efficacy[~nan_map]

RW_lower_bound = vaccine_detail['RW_lower'].to_numpy()
RW_lower_bound = RW_lower_bound[~nan_map]

RW_upper_bound = vaccine_detail['RW_upper'].to_numpy()
RW_upper_bound = RW_upper_bound[~nan_map]

paper_ve_methods = paper_ve_methods[~nan_map]
paper_ci_methods = paper_ci_methods[~nan_map]
references = references[~nan_map]

In [ ]:
plot_compare_reproduced_vaccine_efficacy_scalar_plot(paper_vaccine_efficacy, paper_lower_bound, paper_upper_bound, \
    paper_ve_methods, paper_ci_methods, \
    RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound, save_fig=False)

In [ ]:
plot_compare_reproduced_vaccine_efficacy_error_bar_plot(paper_vaccine_efficacy, paper_lower_bound,
                                                            paper_upper_bound, paper_ve_methods, paper_ci_methods,
                                                            RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound, 
                                                            references, save_fig=False)

In [ ]:
RW_vaccine_efficacy[23]

In [ ]:
print(f'Total number of recalibrated VE: {len(paper_vaccine_efficacy)}')
ve_difference = paper_vaccine_efficacy-RW_vaccine_efficacy
ve_index = np.argsort(ve_difference)

data = {'ve_difference': ve_difference, 'rw_ve': RW_vaccine_efficacy, 'paper_ve_methods': paper_ve_methods, 'paper_ci_methods': paper_ci_methods}
df = pd.DataFrame(data)
print(df[abs(df['ve_difference']) > 3])


In [ ]:
from six import remove_move


def distance_between_efficacy_and_CI_method(paper_vaccine_efficacy, paper_lower_bound,
                                             paper_upper_bound, paper_ve_methods, paper_ci_methods,
                                             RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound, save_fig=True):

    # Efficacy
    plt.figure(figsize=(10, 6))
    unique_paper_ve_methods = np.unique(paper_ve_methods)
    efficacy_difference_dict = {}
    for i, paper_ve_method in enumerate(unique_paper_ve_methods):
        index_map = paper_ve_methods == paper_ve_method
        paper_vaccine_efficacy_tmp = paper_vaccine_efficacy[index_map]
        RW_vaccine_efficacy_tmp = RW_vaccine_efficacy[index_map]
        efficacy_difference = paper_vaccine_efficacy_tmp-RW_vaccine_efficacy_tmp
        efficacy_difference_dict[paper_ve_method] = efficacy_difference


    labels, data = [*zip(*efficacy_difference_dict.items())]  # 'transpose' items to parallel key, value lists
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.xticks(rotation = 30, ha='right')
    plt.grid()
    plt.ylabel('Difference between original vaccine efficacy and \n vaccine efficacy estimated by relative risk (%)')
    if save_fig == True:
        plt.savefig('Difference_efficacy_RW2022.pdf')

    # Remove our 100% efficacy data and paper's 0 CI
    efficacy_remove_map = RW_vaccine_efficacy != 100
    ci_remove_map = paper_lower_bound != 0
    remove_map = efficacy_remove_map & ci_remove_map
    paper_vaccine_efficacy = paper_vaccine_efficacy[remove_map]
    paper_lower_bound = paper_lower_bound[remove_map]
    paper_upper_bound = paper_upper_bound[remove_map]
    paper_ve_methods = paper_ve_methods[remove_map]
    paper_ci_methods = paper_ci_methods[remove_map]
    RW_vaccine_efficacy = RW_vaccine_efficacy[remove_map]
    RW_lower_bound = RW_lower_bound[remove_map]
    RW_upper_bound = RW_upper_bound[remove_map]

    # Lower
    plt.figure(figsize=(10, 6))
    paper_lower_lengths = paper_vaccine_efficacy - paper_lower_bound
    RW_lower_lengths = RW_vaccine_efficacy - RW_lower_bound
    unique_paper_ci_methods = np.unique(paper_ci_methods)
    lower_difference_dict = {}
    for i, paper_ci_method in enumerate(paper_ci_methods):
        lower_index_map = paper_ci_methods == paper_ci_method
        paper_lower_lengths_tmp = paper_lower_lengths[lower_index_map]
        RW_lower_lengths_tmp = RW_lower_lengths[lower_index_map]
        lower_difference = paper_lower_lengths_tmp - RW_lower_lengths_tmp
        lower_difference_dict[paper_ci_method] = lower_difference

    labels, data = [*zip(*lower_difference_dict.items())]  # 'transpose' items to parallel key, value lists
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.xticks(rotation = 30, ha='right')
    plt.grid()
    plt.ylabel('Difference between original lower bound and \n lower bound estimated by Poisson regression (%)')
    if save_fig == True:
        plt.savefig('Difference_upper_RW2022.pdf')
    
    # Upper
    plt.figure(figsize=(10, 6))
    paper_upper_lengths = paper_upper_bound - paper_vaccine_efficacy
    RW_upper_lengths = RW_upper_bound - RW_vaccine_efficacy
    unique_paper_ci_methods = np.unique(paper_ci_methods)
    upper_difference_dict = {}
    for i, paper_ci_method in enumerate(paper_ci_methods):
        upper_index_map = paper_ci_methods == paper_ci_method
        paper_upper_lengths_tmp = paper_upper_lengths[upper_index_map]
        RW_upper_lengths_tmp = RW_upper_lengths[upper_index_map]
        upper_difference = paper_upper_lengths_tmp - RW_upper_lengths_tmp
        upper_difference_dict[paper_ci_method] = upper_difference

    labels, data = [*zip(*upper_difference_dict.items())]  # 'transpose' items to parallel key, value lists
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.xticks(rotation = 30, ha='right')
    plt.grid()
    plt.ylabel('Difference between original upper bound and \n upper bound estimated by Poisson regression (%)')
    if save_fig == True:
        plt.savefig('Difference_lower_RW2022.pdf')
    


In [ ]:
paper_ci_methods[paper_ci_methods=='Events per COVID-19- free person-years'] = 'Events per COVID-19-free person-years'
distance_between_efficacy_and_CI_method(paper_vaccine_efficacy, paper_lower_bound,
                                             paper_upper_bound, paper_ve_methods, paper_ci_methods,
                                             RW_vaccine_efficacy, RW_lower_bound, RW_upper_bound, save_fig=False)